In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
import glob
import os
import time
import cv2
import tensorflow as tf
from tensorflow.keras import layers

In [7]:
!pip install kaggle

In [8]:
import os
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi

# Set up Kaggle API credentials
os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)
!cp kaggle.json ~/.kaggle/
os.chmod(os.path.expanduser('~/.kaggle/kaggle.json'), 0o600)

# Initialize Kaggle API
api = KaggleApi()
api.authenticate()

# Download the dataset
dataset = 'jessicali9530/celeba-dataset'
api.dataset_download_files(dataset, path='./', unzip=True)

# Unzip the downloaded file if needed
with zipfile.ZipFile('./celeba-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('./dataset')

print("Dataset downloaded and extracted successfully.")


'cp' is not recognized as an internal or external command,
operable program or batch file.


FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:\\Users\\prasa/.kaggle/kaggle.json'

In [5]:
import zipfile
with zipfile.ZipFile('./celeba-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('./dataset')


BadZipFile: File is not a zip file

In [ ]:
import cv2
import imghdr
data_dir = ""
import numpy as np
from matplotlib import pyplot as plt
data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    label_mode=None,  # No labels are inferred
    image_size=(64,64),  # Resize images to a specific size
    batch_size=128  # Number of images to return in each batch
)


In [ ]:
for images in data.take(1):
    print(images.shape)
    plt.figure(figsize=(10, 10))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.axis("off")
    plt.show()

In [ ]:
data = data.map(lambda x: (x/255))
data.as_numpy_iterator().next().shape

In [ ]:
# train_size = int(len(data)*.7)
# val_size = int(len(data)*.2)
# test_size = int(len(data)*.1)
# train_data = data.take(train_size)
# val_data = data.skip(train_size).take(val_size)
# test_data = data.skip(train_size+val_size).take(test_size)

VAE

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (Reshape, Conv2DTranspose, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input, Lambda)
from tensorflow.keras.optimizers import Adam

In [11]:
input_shape = (64, 64, 3)  
latent_dim = 128  

In [12]:
#encoder

inputs = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(64, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)

z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)


def reparamaetrization(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(reparamaetrization, output_shape=(latent_dim,))([z_mean, z_log_var])
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')



In [14]:
encoder.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 64, 64,    │        896 │ input_layer_3[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 32, 32,    │     18,496 │ conv2d_12[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 32, 32,    │     36,928 │ conv2d_13[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 16, 16,    │     73,856 │ conv2d_14[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 32768)     │          0 │ conv2d_15[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 256)       │  8,388,864 │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 128)       │     32,896 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 128)       │     32,896 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 128)       │          0 │ dense_10[0][0],   │
│                     │                   │            │ dense_11[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8,584,832 (32.75 MB)

 Trainable params: 8,584,832 (32.75 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
#decoder
latent_inputs = Input(shape=(latent_dim,))
x = Dense(16 * 16 * 128, activation='relu')(latent_inputs)
x = Reshape((16, 16, 128))(x)
x = Conv2DTranspose(128, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
x = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
outputs = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)

decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 32768)          │     4,227,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 32, 32, 128)    │       147,584 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 32, 32, 64)     │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 64, 64, 64)     │        36,928 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_3              │ (None, 64, 64, 32)     │        18,464 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_4              │ (None, 64, 64, 3)      │           867 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,504,707 (17.18 MB)

 Trainable params: 4,504,707 (17.18 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
vae_input = Input(input_shape, name="vae_input")
z,_,_ = encoder(vae_input)
output = decoder(z)
vae = Model(vae_input, output, name="vae")
vae.summary()
     

Model: "vae"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vae_input (InputLayer)          │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Functional)            │ [(None, 128), (None,   │     8,584,832 │
│                                 │ 128), (None, 128)]     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Functional)            │ (None, 64, 64, 3)      │     4,504,707 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,089,539 (49.93 MB)

 Trainable params: 13,089,539 (49.93 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:

def custom_loss(y_true,y_pred,mean,log_var):

  loss_rec = tf.reduce_mean(tf.reduce_sum(tf.keras.losses.binary_crossentropy(y_true,y_pred), axis = (1,2)))

  loss_reg = -0.5 * (1 + log_var - tf.square(mean) - tf.exp(log_var))
  
  return loss_rec+tf.reduce_mean(tf.reduce_sum(loss_reg, axis=1))

In [ ]:
OPTIMIZER = Adam(learning_rate = 1e-3)
class VAE(tf.keras.Model):
    def __init__(self, encoder,decoder):
        super(VAE, self).__init__()
        self.encoder=encoder
        self.decoder=decoder
        self.loss_tracker=tf.keras.metrics.Mean(name='loss')
    @property
    def metrics(self):
        return [self.loss_tracker]
        
    def train_step(self,train_batch):
        with tf.GradientTape() as recorder:
            z,mean,log_var = encoder(train_batch)
            y_pred = decoder(z)
            y_true = train_batch
            loss = custom_loss(y_true,y_pred, mean, log_var)
        
        partial_derivatives = recorder.gradient(loss,self.trainable_weights)
        OPTIMIZER.apply_gradients(zip(partial_derivatives, self.trainable_weights))
        self.loss_tracker.update_state(loss)
        return {'loss':self.loss_tracker.result()}

In [ ]:
model=vae(encoder,decoder)
model.compile(optimizer=OPTIMIZER, metrics=['accuracy'])

logdir='logs1'

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
model.fit(data,epochs=50,batch_size=128,callbacks=[tensorboard_callback])

In [ ]:
num_images = 10  
random_latent_vectors = np.random.normal(size=(num_images, latent_dim))
generated_images = decoder.predict(random_latent_vectors)
generated_images = (generated_images * 255).astype(np.uint8)
plt.figure(figsize=(20, 4))
for i in range(num_images):
    ax = plt.subplot(1, num_images, i + 1)
    plt.imshow(generated_images[i])
    plt.axis("off")
plt.show()

In [ ]:
from tensorflow.keras.models import load_model

# Save the model
model.save(os.path.join('models', 'vae_facegen.h5'))

VAE+DCGAN 

In [ ]:
from tensorflow.keras.layers import LeakyReLU

# Discriminator model
discriminator_input = Input(shape=input_shape, name='discriminator_input')
x = Conv2D(32, 3, strides=2, padding='same')(discriminator_input)
x = LeakyReLU(alpha=0.2)(x)
x = Conv2D(64, 3, strides=2, padding='same')(x)
x = LeakyReLU(alpha=0.2)(x)
x = Flatten()(x)
x = Dense(128)(x)
x = LeakyReLU(alpha=0.2)(x)
x = Dense(1, activation='sigmoid')(x)
discriminator = Model(discriminator_input, x, name='discriminator')
discriminator.summary()


In [ ]:
batch_size=128 
class VAEGAN(tf.keras.Model):
    def __init__(self, encoder,decoder,discriminator):
        super(VAE, self).__init__()
        self.encoder=encoder
        self.decoder=decoder
        self.discriminator=discriminator
        self.loss_tracker=tf.keras.metrics.Mean(name='loss')
        self.vae_optimizer = tf.keras.optimizers.Adam(lr=0.0005)
        self.discriminator_optimizer = tf.keras.optimizers.Adam(lr=0.0005)
        self.generator_optimizer = tf.keras.optimizers.Adam(lr=0.0005)
    @property
    def metrics(self):
        return [self.loss_tracker]
        
    def train_step(self,train_batch):
        real_images=train_batch
        fake_images=decoder.predict(train_batch)
        with tf.GradientTape() as recorder:
            z,mean,log_var = encoder(train_batch)
            y_pred = decoder(z)
            y_true = train_batch
            loss = custom_loss(y_true,y_pred, mean, log_var)
        
        partial_derivatives = recorder.gradient(loss,self.trainable_weights)
        self.vae_optimizer.apply_gradients(zip(partial_derivatives, self.trainable_weights))
        with tf.GradientTape() as d_tape:
            yhat_real = self.discriminator(real_images, training=True) 
            yhat_fake = self.discriminator(fake_images, training=True)
            yhat_realfake = tf.concat([yhat_real, yhat_fake], axis=0)
            y_realfake = tf.concat([tf.zeros_like(yhat_real), tf.ones_like(yhat_fake)], axis=0)
            noise_real = 0.15 * tf.random.uniform(tf.shape(yhat_real))
            noise_fake = -0.15 * tf.random.uniform(tf.shape(yhat_fake))
            y_realfake += tf.concat([noise_real, noise_fake], axis=0)
            total_d_loss = self.d_loss(y_realfake, yhat_realfake)
        
        dgrad = d_tape.gradient(total_d_loss, self.discriminator.trainable_variables) 
        self.discriminator_optimizer.apply_gradients(zip(dgrad, self.discriminator.trainable_variables))
        with tf.GradientTape() as g_tape:
            gen_imgs = self.decoder(tf.random.normal((batch_size, 128)), training=True)
            predicted_labels = self.discriminator(gen_imgs, training=False)
            total_g_loss = self.g_loss(tf.zeros_like(predicted_labels), predicted_labels)
        
        ggrad = g_tape.gradient(total_g_loss, self.decoder.trainable_variables)
        self.generator_optimizer.apply_gradients(zip(ggrad, self.decoder.trainable_variables))
        self.loss_tracker.update_state(loss+total_d_loss+total_g_loss)
        return {"loss": self.loss_tracker.result()}

In [ ]:
model=VAEGAN(encoder,decoder,discriminator)
model.compile(metrics=['accuracy'])
logdir='logs1'

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
model.fit(data,epochs=10000,batch_size=128,callbacks=[tensorboard_callback])
